<a href="https://colab.research.google.com/github/dkm4/Image-classification-model-for-environment-factors/blob/main/Image_classification_model_for_environment_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connect google drive


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#check the dataset pathway


In [2]:
dataset_path = '/content/drive/MyDrive/environment dataset'
print(f"data set pathway is {dataset_path}")

data set pathway is /content/drive/MyDrive/environment dataset


#import

In [3]:
import os
import glob
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
print(torch.__version__)
print(torchvision.__version__)


2.2.1+cu121
0.17.1+cu121


#device

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#check the directory

In [5]:
print(os.listdir(dataset_path))
print(os.path.join(dataset_path,'train'))

['train', 'test']
/content/drive/MyDrive/environment dataset/train


#Connect the pathway to train, test data

In [6]:
train_data_location = os.path.join(dataset_path,'train')
test_data_location = os.path.join(dataset_path,'test')
print(f"my train data loction is: {train_data_location}")
print(f"my test data loction is: {test_data_location}")

my train data loction is: /content/drive/MyDrive/environment dataset/train
my test data loction is: /content/drive/MyDrive/environment dataset/test


#Build dataset and dataloader by connecting pathway

In [7]:
#you take in the pathway to the train/test as an input and it returns the classes and the index of the classes inside the train/test dataset an output
def find_classes(path_to_data):
  classes = sorted(os.listdir(path_to_data))
  class_to_idx = {c:i for i, c in enumerate(classes)}
  return classes, class_to_idx


In [8]:
#takes in the train/test pathway as an input and return all the image in train/test
def give_me_all_images_path(data_location):
  return glob.glob(os.path.join(data_location, '*', '*.*'), recursive=True)

#give_me_all_images_path(train_data_location)

In [9]:
def pathToClass(path):
  dir_path = os.path.dirname(path)
  return os.path.basename(dir_path)


In [10]:
from torch.utils.data import Dataset
class ImageFolderCustom(Dataset):
  def __init__(self, dataset_path, transform):
    self.dataset_path = dataset_path
    self.transform    = transform
    self.classes, self.class_to_idx = find_classes(dataset_path)
    self.paths = give_me_all_images_path(dataset_path)

  def __len__(self):
    return len(self.paths)

  def __getitem__(self, idx):
    path = self.paths[idx]
    img = Image.open(path)
    my_class = pathToClass(path)
    class_label = self.class_to_idx[my_class]
    if self.transform:
      return self.transform(img), class_label
    else:
      return img, class_label

In [11]:
train_transform = transforms.Compose([
  transforms.Resize((32,32)),
  transforms.RandomRotation(10),
     transforms.ToTensor(),
 ])
test_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
 ])

In [12]:
train_dataset = ImageFolderCustom(train_data_location, transform = train_transform)
test_dataset = ImageFolderCustom(test_data_location, transform = test_transform)

In [13]:
BATCH_SIZE = 8
NUM_CLASSES = 4
train_data_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_data_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)



In [14]:
for batch_idx, i in enumerate(test_data_loader):
  print(f"batch:{batch_idx+1} and number of points are {len(i[0])}")

batch:1 and number of points are 8
batch:2 and number of points are 4


#build the NN architecture

In [15]:
# class FashionNN(nn.Module):
#   def __init__(self, in_features, out_features, hidden_layer=20):
#     super().__init__()
#     self.layer = nn.Sequential(
#                               nn.Flatten(),
#                               nn.Linear(in_features=in_features, out_features=hidden_layer),
#                                nn.ReLU(),
#                                nn.Linear(in_features=hidden_layer, out_features=out_features),
#                                )
#   def forward(self, x):
#     return self.layer(x)
# model = FashionNN(3*32*32, NUM_CLASSES)

class ConvNN(nn.Module):
  def __init__(self, in_channels=3, out_channels=NUM_CLASSES, hidden_channels=10):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, padding=0, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=0, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
    )


    self.fcnn = nn.Sequential(
        nn.Flatten(),
        nn.Linear(6*6*32, out_channels)
    )

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = self.fcnn(x)
    return x

model = ConvNN(in_channels=3, out_channels=NUM_CLASSES)

In [16]:
rand_tensor = torch.rand(10,3,32,32)
result_tensor = model(rand_tensor)
print(f"shape of result tensor are: {result_tensor.shape}")

shape of result tensor are: torch.Size([10, 4])


#build loss function and optimizer

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

#build different def function(acc, train step, test step)

In [18]:
def acc_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/ len(y_true))*100.0
  return acc

def train_step(model, training_dataloader, loss_fn, optimizer, device):
  train_loss = 0
  train_acc = 0
  model.train()
  for batch_idx, item in enumerate(training_dataloader):
    X_batch, y_batch = item
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)
    model.to(device)
    logits = model(X_batch)
    training_loss = loss_fn(logits, y_batch)
    train_loss += training_loss
    y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
    training_acc = acc_fn(y_batch, y_preds)
    train_acc+=training_acc
    optimizer.zero_grad()
    training_loss.backward()
    optimizer.step()
  print(f"My training loss: {train_loss/len(training_dataloader)} Training acc:{train_acc/len(training_dataloader)}")


def test_step(model, testing_dataloader, loss_fn, device):
  test_loss = 0
  test_acc  = 0
  model.eval()
  with torch.no_grad():
    for batch_idx, item in enumerate(testing_dataloader):
      X_batch, y_batch = item
      X_batch = X_batch.to(device)
      y_batch = y_batch.to(device)
      model.to(device)
      logits = model(X_batch)
      testing_loss = loss_fn(logits, y_batch)
      test_loss += testing_loss
      y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
      testing_acc = acc_fn(y_batch, y_preds)
      test_acc+=testing_acc
  print(f"My testing loss: {test_loss/len(testing_dataloader)} Training acc:{test_acc/len(testing_dataloader)}")

#train the model

In [19]:
from tqdm.auto import tqdm
epochs = 500
for epoch in tqdm(range(epochs)):
  print(f"My epoch {epoch}\n-----------")
  train_step(model, train_data_loader, loss_fn, optimizer, device)
  test_step(model, test_data_loader, loss_fn, device)
  print("\n\n")

  0%|          | 0/500 [00:00<?, ?it/s]

My epoch 0
-----------
My training loss: 1.381996989250183 Training acc:28.75
My testing loss: 1.3863052129745483 Training acc:18.75



My epoch 1
-----------
My training loss: 1.3595134019851685 Training acc:31.25
My testing loss: 1.3886258602142334 Training acc:18.75



My epoch 2
-----------
My training loss: 1.3331667184829712 Training acc:35.0
My testing loss: 1.3861875534057617 Training acc:18.75



My epoch 3
-----------
My training loss: 1.3149670362472534 Training acc:36.25
My testing loss: 1.386887788772583 Training acc:18.75



My epoch 4
-----------
My training loss: 1.3009049892425537 Training acc:36.25
My testing loss: 1.3912020921707153 Training acc:18.75



My epoch 5
-----------
My training loss: 1.2983735799789429 Training acc:32.5
My testing loss: 1.4013195037841797 Training acc:18.75



My epoch 6
-----------
My training loss: 1.2699236869812012 Training acc:36.25
My testing loss: 1.4055657386779785 Training acc:18.75



My epoch 7
-----------
My training loss: 1.25